In [422]:
import numpy as np 
import pandas as pd
from itertools import *
from mip import *
from datetime import *
import re
from decimal import Decimal

In [291]:
def montar_arquivo(filename):
    """Função para ler o arquivo e gerar as estruturas de dados que serão consumidas pelo
    modelo de otimização"""

    conteudo = []

    with open(filename)  as f:
        for line in f:
            conteudo.append(line.strip('\n'))

    header = conteudo[0].split(' ')
    m = np.int32(header[1])
    n = np.int32(header[2])
    raw_data = {}
    for i in range(1, len(conteudo)):    
        raw_data[i] = conteudo[i].split(' ')

    dados = {}
    conjunto = []
    id = 0
    for k in raw_data.keys():
        aux = []
        for x in raw_data[k]:       
            if x != "" and x != 'capacity':
                aux.append(float(x))
            elif x == 'capacity':
                aux.append(-1)
        conjunto.append(aux)                
                
        
    demand_raw = []
    cost_raw = []
    count = 0
    conjunto = list(chain(*conjunto))
    for i in range(2*m, len(conjunto)):
        if count % (m+1) == 0:
            demand_raw.append(conjunto[i])
        else:
            cost_raw.append(conjunto[i])   
        count += 1         

   
    dict_capacity = {'a':[8000,10000,12000,14000], 'b': [5000,6000,7000,8000],
                    'c':[5000, 5750, 6500, 7250]}

    text = r'cap(\w)'
    letra = re.findall(text, filename)

    if letra[0] in ['a','b','c']:
        capacity = [dict_capacity[letra[0]][0] for i in range(m)]

    else:  
        capacity = [conjunto[i] for i in range(2*m) if i%2==0]
    cost_fixed = [conjunto[i] for i in range(2*m) if i%2==1]


    demand = demand_raw
    cost_prep = cost_raw

    cost = {}
    id = 0
    for i in range(n):
        aux = []
        for j in range(m):
            #print(i,j, id)    
            if id < (n*m):
                aux.append(cost_prep[id])         
            id += 1    
        cost[i] = aux
    return demand, capacity, cost, cost_fixed  

In [504]:
def solve_model(demand, capacity, cost, cost_fixed, filename, cut_plan = 'AUTOMATIC'):

    now = datetime.now()
    now = now.strftime("%Y-%m-%d_%H_%M_%S")

    cuts_types = {
              None: 0,
              'AUTOMATIC': -1, 
             'GOMORY': CutType.GOMORY,
              'KNAPSACK_COVER': CutType.KNAPSACK_COVER,
              'CLIQUE': CutType.CLIQUE,
              'LIFT': CutType.LIFT_AND_PROJECT}
    
    cut_code = cuts_types[cut_plan]

    model = Model()

    m = len(capacity)

    n = len(demand)

    model.store_search_progress_log = True

    x = [[model.add_var(name = f'x{i}{j}', lb = 0, ub = 1) for j in range(m)] for i in range(n)]

    y = [model.add_var(name = f'y{j}', var_type = BINARY) for j in range(m)]

    model.objective = xsum(y[j]*cost_fixed[j] for j in range(m)) +  xsum(x[i][j]*cost[i][j] for i in range(n) for j in range(m))

    for j in range(m):
        model += xsum(x[i][j]*demand[i] for i in range(n))<= capacity[j]*y[j]

    for i in range(n):
            model += xsum(x[i][j]*demand[i] for j in range(m))>= demand[i]

    for j in range(m):
        for i in range(n):
                model+= x[i][j]<=demand[i]*y[j]

    if cut_code == 0:
         model.cuts = 0    
    elif cut_plan in ['GOMORY', 'KNAPSACK_COVER', 'CLIQUE', 'LIFT']:
        if model.solver_name.lower() == "cbc":
            model.optimize(relax = True)
            cp = model.generate_cuts([CutType.GOMORY])
            if cp.cuts:
                model += cp
    else:
         model.cuts = -1                

    file = filename[8:13]            

    setting = f"{m*n}_{cut_plan}," 
    log =  f"log_{m*n}_{cut_plan}_{now}"                  
               

    model.write('capacitaded_problem.lp')

    model.optimize(max_seconds = 1500)
    progress = model.search_progress_log
    progress.instance = setting
    progress.write(f'./logs/{log}')
    results = model.objective_value
    return results
    

In [507]:
#filename = '../data/cap134.txt'
#filename = '../data/cap41.txt'
filename = '../data/cap104.txt'
#filename = '../data/capa.txt'

In [508]:
resultados = {}
demand, capacity, cost, cost_fixed  = montar_arquivo(filename)
for i in range(3):
    for cut_name in [None, 'AUTOMATIC', 'GOMORY', 'KNAPSACK_COVER']:
        resultados[cut_name] =  solve_model(demand, capacity, cost, cost_fixed, filename, cut_plan = cut_name) 

In [369]:
caminho = './logs/log_2500_AUTOMATIC_2023-07-18_22_38_40.plog'

In [459]:
conteudo = []

with open(caminho)  as f:
    for line in f:
        conteudo.append(line.strip(','))

In [460]:
lista = conteudo[0].split(',')

In [492]:
tempo = {'Instancia':[],
        'Duracao': []}

limites = {'Instancia':[],        
        'Lower_Bound': [],
         'Upper_Bound': []}

In [308]:
model = Model()
#model.cuts = 0

m = len(capacity)

n = len(demand)

model.store_search_progress_log = True

x = [[model.add_var(name = f'x{i}{j}', lb = 0, ub = 1) for j in range(m)] for i in range(n)]
y = [model.add_var(name = f'y{j}', var_type = BINARY) for j in range(m)]

model.objective = xsum(y[j]*cost_fixed[j] for j in range(m)) +  xsum(x[i][j]*cost[i][j] for i in range(n) for j in range(m))

for j in range(m):
    model += xsum(x[i][j]*demand[i] for i in range(n))<= capacity[j]*y[j]

for i in range(n):
    model += xsum(x[i][j]*demand[i] for j in range(m))>= demand[i]

for j in range(m):
    for i in range(n):
        model+= x[i][j]<=demand[i]*y[j]

model.optimize(relax = True)

cp = model.generate_cuts([CutType.GOMORY])
        

model.write('capacitaded_problem.lp')

model.optimize()
#progress = model.search_progress_log
#progress.instance = 'teste'
#progress.write('log')

<OptimizationStatus.OPTIMAL: 0>

In [509]:
resultados

{None: 928941.7500000001,
 'AUTOMATIC': 928941.7500000001,
 'GOMORY': 928941.7500000001,
 'KNAPSACK_COVER': 928941.7500000001}